In [1]:
import sklearn
import ast
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
from sklearn.tree import plot_tree

#Linear model
from sklearn.linear_model import LinearRegression

#KNN
from sklearn.neighbors import KNeighborsRegressor

#RF
from sklearn.ensemble import RandomForestRegressor

#SVR
from sklearn.svm import SVR

#Metrics
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error

#Feature selection
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

#Lasso
from sklearn.linear_model import Lasso

#feature selection
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

#Ridge
from sklearn.linear_model import Ridge

#Gradient Boosting 
#lightgbm
import lightgbm as lgb
#catboost
import catboost as ctb
#xgboost
import xgboost as xgb
import matplotlib as mpl

#Neural network
import math
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler


In [2]:
data = pd.read_csv('final_data_without_prefs_and_with_HAMADA_factors.csv')
data_ret = pd.read_csv('data_ret.csv')
data_ret.drop(columns=['Unnamed: 0'], inplace=True)
#data_ret.set_index('Date', inplace=True)
data.set_index('Date', inplace = True)
macro_data = pd.read_csv('macro_data.csv')
macro_data.set_index('Date', inplace = True)

ml_results_best_params = pd.read_csv('ML_best_params_ever.csv')
ml_data_1 = pd.read_csv('MSE_result_ever.csv')
ml_data = pd.read_csv('R2_result_ever.csv')

ml_data.set_index('share', inplace=True)
ml_data_1.set_index('share', inplace=True)
ml_results_best_params.set_index('share', inplace=True)

# Подгрузим данные
df = pd.read_csv('russian_data_without_prefs.csv')

# Удалим все дни где дневная доходность всех бумаг равна нулю (праздники и выходные)
df.drop(df[df.filter(regex='return_').sum(axis = 1) == 0].index, axis=0, inplace=True)
df.set_index('Date', inplace = True)

# Lasso

In [34]:
lag_autolag_return_30 = 30
lag_autolag_return_60 = 60
lag_autolag_return_90 = 90
lag_30 = 30
lag_60 = 60
lag_90 = 90


Long_returns_lasso = []
Short_returns_lasso = []

Long_10_lasso = []
Short_10_lasso = []
Hold_lasso = []


window = 29
gap_10 = 0.01

for name in data_ret.columns[2:].drop(['return_PLZL_RM_Equity', 'return_RSTI_RM_Equity','return_MSRS_RM_Equity']):

    df1 = data[[name, 'Hamada_beta_MRP_for_{}'.format(name[7:11]), 'risk_free_rate_based_on_g_curve',
                 'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
                 'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]),
                 'MOM']].dropna()
    df_autolag = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)})
    df_autolag = pd.DataFrame(df_autolag['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)].shift(lag_autolag_return_30).dropna())

    df_autolag_2 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)})
    df_autolag_2 = pd.DataFrame(df_autolag_2['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)].shift(lag_autolag_return_60).dropna())

    df_autolag_3 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)})
    df_autolag_3 = pd.DataFrame(df_autolag_3['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)].shift(lag_autolag_return_90).dropna())

    df2 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_30).dropna()



    df2.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
                          'MOM':'MOM_lag_{}_days'.format(lag_30)}, inplace = True)

    result = pd.concat([df1, df2], axis=1, join="inner")

    df3 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_60).dropna()

    df3.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
                          'MOM':'MOM_lag_{}_days'.format(lag_60)}, inplace = True)

    result = pd.concat([result, df3], axis=1, join="inner")

    df4 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_90).dropna()

    df4.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90), 
                          'MOM':'MOM_lag_{}_days'.format(lag_90)}, inplace = True)

    result = pd.concat([result, df4], axis=1, join="inner")

    result = pd.merge(result, macro_data, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_2, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_3, left_index=True, right_index=True)
    result['{}_Equity_premium'.format(name[7:14])] = result[name] - result['risk_free_rate_based_on_g_curve']

    share = '{}_Equity_premium'.format(name[7:14])

    dta = result.dropna()

    X_train = dta.loc[:'2017-01-30',:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    X_test = dta.loc['2017-01-30':,:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    y_train = dta.loc[:'2017-01-30',:][share]
    y_test = dta.loc['2017-01-30':,:][share]

    lasso = Lasso(alpha = ast.literal_eval(ml_results_best_params.loc[share,'LASSO_best_params'])['alpha'],
                  normalize=ast.literal_eval(ml_results_best_params.loc[share,'LASSO_best_params'])['normalize'],
                  tol=ast.literal_eval(ml_results_best_params.loc[share,'LASSO_best_params'])['tol'])

    lasso.fit(X_train, y_train)
    y_pred_1 = lasso.predict(X_train.iloc[-1:])

    # фактическая больше модельной
    if y_train[-1] -  y_pred_1 > gap_10:
        Short_10_lasso.append(name)
    # модельная больше фактической
    elif y_pred_1 - y_train[-1] > gap_10:
        Long_10_lasso.append(name)
    elif abs(y_pred_1 - y_train[-1]) < gap_10:
            Hold_lasso.append(name)
    else:
        continue

Recomends_lasso = pd.DataFrame({'Short':pd.Series(Short_10_lasso), 
                                  'Hold':pd.Series(Hold_lasso),
                                  'Long':pd.Series(Long_10_lasso)})

Recomends_lasso['Short'] = Recomends_lasso['Short'].astype(str).str[7:11]
Recomends_lasso['Hold'] = Recomends_lasso['Hold'].astype(str).str[7:11]
Recomends_lasso['Long'] = Recomends_lasso['Long'].astype(str).str[7:11]
Recomends_lasso.to_excel('Great_recom_bear_lasso.xlsx', sheet_name='recom')


# CAPM

In [35]:
lag_autolag_return_30 = 30
lag_autolag_return_60 = 60
lag_autolag_return_90 = 90
lag_30 = 30
lag_60 = 60
lag_90 = 90


Long_returns_capm = []
Short_returns_capm = []

Long_10_capm = []
Short_10_capm = []
Hold_capm = []

window = 29
gap_10 = 0.01

for name in data_ret.columns[2:].drop(['return_PLZL_RM_Equity', 'return_RSTI_RM_Equity','return_MSRS_RM_Equity']):

    df1 = data[[name, 'Hamada_beta_MRP_for_{}'.format(name[7:11]), 'risk_free_rate_based_on_g_curve',
                 'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
                 'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]),
                 'MOM']].dropna()
    df_autolag = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)})
    df_autolag = pd.DataFrame(df_autolag['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)].shift(lag_autolag_return_30).dropna())

    df_autolag_2 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)})
    df_autolag_2 = pd.DataFrame(df_autolag_2['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)].shift(lag_autolag_return_60).dropna())

    df_autolag_3 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)})
    df_autolag_3 = pd.DataFrame(df_autolag_3['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)].shift(lag_autolag_return_90).dropna())

    df2 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_30).dropna()



    df2.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
                          'MOM':'MOM_lag_{}_days'.format(lag_30)}, inplace = True)

    result = pd.concat([df1, df2], axis=1, join="inner")

    df3 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_60).dropna()

    df3.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
                          'MOM':'MOM_lag_{}_days'.format(lag_60)}, inplace = True)

    result = pd.concat([result, df3], axis=1, join="inner")

    df4 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_90).dropna()

    df4.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90), 
                          'MOM':'MOM_lag_{}_days'.format(lag_90)}, inplace = True)

    result = pd.concat([result, df4], axis=1, join="inner")

    result = pd.merge(result, macro_data, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_2, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_3, left_index=True, right_index=True)
    result['{}_Equity_premium'.format(name[7:14])] = result[name] - result['risk_free_rate_based_on_g_curve']

    share = '{}_Equity_premium'.format(name[7:14])

    dta = result.dropna()

    X_train = dta.loc[:'2017-01-30',:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    X_test = dta.loc['2017-01-30':,:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    y_train = dta.loc[:'2017-01-30',:][share]
    y_test = dta.loc['2017-01-30':,:][share]

    model = LinearRegression(fit_intercept=False)
    model.fit(X_train.iloc[:,0:1], y_train)

    y_pred_1 = model.predict(X_train.iloc[-1:,0:1])

    # фактическая больше модельной
    if y_train[-1] -  y_pred_1 > gap_10:
        Short_10_capm.append(name)
    # модельная больше фактической
    elif y_pred_1 - y_train[-1] > gap_10:
        Long_10_capm.append(name)
    elif abs(y_pred_1 - y_train[-1]) < gap_10:
        Hold_capm.append(name)
    else:
        continue




Recomends_capm = pd.DataFrame({'Short':pd.Series(Short_10_capm), 
                                  'Hold':pd.Series(Hold_capm),
                                  'Long':pd.Series(Long_10_capm)})

Recomends_capm['Short'] = Recomends_capm['Short'].astype(str).str[7:11]
Recomends_capm['Hold'] = Recomends_capm['Hold'].astype(str).str[7:11]
Recomends_capm['Long'] = Recomends_capm['Long'].astype(str).str[7:11]
Recomends_capm.to_excel('Great_recom_bear_capm.xlsx', sheet_name='recom')


       


# Fama-French

In [36]:
lag_autolag_return_30 = 30
lag_autolag_return_60 = 60
lag_autolag_return_90 = 90
lag_30 = 30
lag_60 = 60
lag_90 = 90


Long_returns_ff = []
Short_returns_ff = []

Long_10_ff = []
Short_10_ff = []
Hold_ff = []

window = 29
gap_10 = 0.01

for name in data_ret.columns[2:].drop(['return_PLZL_RM_Equity', 'return_RSTI_RM_Equity','return_MSRS_RM_Equity']):

    df1 = data[[name, 'Hamada_beta_MRP_for_{}'.format(name[7:11]), 'risk_free_rate_based_on_g_curve',
                 'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
                 'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]),
                 'MOM']].dropna()
    df_autolag = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)})
    df_autolag = pd.DataFrame(df_autolag['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_30)].shift(lag_autolag_return_30).dropna())

    df_autolag_2 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)})
    df_autolag_2 = pd.DataFrame(df_autolag_2['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_60)].shift(lag_autolag_return_60).dropna())

    df_autolag_3 = pd.DataFrame(data[name] - data['risk_free_rate_based_on_g_curve']).rename(columns = {0:'{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)})
    df_autolag_3 = pd.DataFrame(df_autolag_3['{}_premium_lag_{}_days'.format(name[7:], lag_autolag_return_90)].shift(lag_autolag_return_90).dropna())

    df2 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_30).dropna()



    df2.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
                          'MOM':'MOM_lag_{}_days'.format(lag_30)}, inplace = True)

    result = pd.concat([df1, df2], axis=1, join="inner")

    df3 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_60).dropna()

    df3.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
                          'MOM':'MOM_lag_{}_days'.format(lag_60)}, inplace = True)

    result = pd.concat([result, df3], axis=1, join="inner")

    df4 = data[['Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
            'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 'MOM']].shift(lag_90).dropna()

    df4.rename(columns = {'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
                          'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]):'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90), 
                          'MOM':'MOM_lag_{}_days'.format(lag_90)}, inplace = True)

    result = pd.concat([result, df4], axis=1, join="inner")

    result = pd.merge(result, macro_data, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_2, left_index=True, right_index=True)
    result = pd.merge(result, df_autolag_3, left_index=True, right_index=True)
    result['{}_Equity_premium'.format(name[7:14])] = result[name] - result['risk_free_rate_based_on_g_curve']

    share = '{}_Equity_premium'.format(name[7:14])

    dta = result.dropna()

    X_train = dta.loc[:'2017-01-30',:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    X_test = dta.loc['2017-01-30':,:][['Hamada_beta_MRP_for_{}'.format(name[7:11]),
       '{}_premium_lag_{}_days'.format(name[7:], lag_30),
       'Hamada_d_mcap_ratio_SMB_for_{}'.format(name[7:11]),
       'Hamada_d_mcap_ratio_HML_for_{}'.format(name[7:11]), 
       'MOM',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_30),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_30), 
       'MOM_lag_30_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_60),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_60), 
       'MOM_lag_60_days',
       'Hamada_d_mcap_ratio_SMB_for_{}_lag_{}_days'.format(name[7:11], lag_90),
       'Hamada_d_mcap_ratio_HML_for_{}_lag_{}_days'.format(name[7:11], lag_90),  
       'MOM_lag_90_days',
       'dollar_30d_return', 'wti_30d_return']]
    y_train = dta.loc[:'2017-01-30',:][share]
    y_test = dta.loc['2017-01-30':,:][share]

    model = LinearRegression(fit_intercept=False)
    model.fit(X_train.iloc[:,[0, 2, 3]], y_train)

    y_pred_1 = model.predict(X_train.iloc[-1:,[0, 2, 3]])

    # фактическая больше модельной
    if y_train[-1] -  y_pred_1 > gap_10:
        Short_10_ff.append(name)
    # модельная больше фактической
    elif y_pred_1 - y_train[-1] > gap_10:
        Long_10_ff.append(name)
    elif abs(y_pred_1 - y_train[-1]) < gap_10:
        Hold_ff.append(name)
    else:
        continue

        
Recomends_FF = pd.DataFrame({'Short':pd.Series(Short_10_ff), 
                                  'Hold':pd.Series(Hold_ff),
                                  'Long':pd.Series(Long_10_ff)})
        
Recomends_FF['Short'] = Recomends_FF['Short'].astype(str).str[7:11]
Recomends_FF['Hold'] = Recomends_FF['Hold'].astype(str).str[7:11]
Recomends_FF['Long'] = Recomends_FF['Long'].astype(str).str[7:11]

Recomends_FF.to_excel('Great_recom_bear_ff.xlsx', sheet_name='Рекомы_Фама_Френч')
